In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

from sklearn import svm,preprocessing

from sklearn.model_selection import RandomizedSearchCV

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

import scoring as score # para hacer los reportes de puntajes

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
properati = pd.read_csv('datos/properati_final.csv')

<IPython.core.display.Javascript object>

In [8]:
#paso features con strings a numericos
le_barrio = preprocessing.LabelEncoder()
barrios=properati['state_name']
le_barrio.fit(barrios)
properati['state_name'] = le_barrio.transform(barrios)
#test_samples['state_name']=le_barrio.transform(test_samples['state_name'])

le_zona = preprocessing.LabelEncoder()
#zona=pd.concat([properati['place_name'],test_samples['place_name']])
zona=properati['place_name']
le_zona.fit(zona)
properati['place_name'] = le_zona.transform(properati['place_name'])
#test_samples['place_name'] = le_zona.transform(test_samples['place_name'])


le_tipo = preprocessing.LabelEncoder()
tipos_prop=properati['property_type']
le_tipo.fit(tipos_prop)
properati['property_type'] = le_tipo.transform(tipos_prop)
#test_samples['property_type']=le_tipo.transform(test_samples['property_type'])

In [9]:
clf = svm.SVR()
scaler = preprocessing.StandardScaler()
scaler2= preprocessing.MinMaxScaler()

# Random Search

In [10]:
#preparo set de datos
X = zip(properati['surface_total_in_m2'],\
        properati['surface_covered_in_m2'],properati["property_type"],properati['state_name'],properati['place_name'])
y = properati['price_aprox_usd']

X=scaler.fit_transform(X,y)
X=scaler2.fit_transform(X,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


In [11]:
C_range = np.arange(0.1,1000,0.1)
gamma_range = np.arange(10,20,1)

## Por defecto usa kernel rbf

In [ ]:
%%notify

param_grid ={"gamma": gamma_range, "C": C_range}

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

random_search = RandomizedSearchCV(clf,param_distributions=param_grid,n_iter=20,cv=5,n_jobs=-1)
start = time()
random_search.fit(X_train, y_train)

print("RandomizedSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(random_search.cv_results_['params'])))
score.report_single(random_search.cv_results_)

In [ ]:
mejor = random_search.best_estimator_

In [ ]:
properati['svm']=random_search.predict(X,y) 